# Data Preprocessing for multi-class Classification 
# in the domain of TCGA-STAD (The Cancer Genome Atlas Stomach Adenocarcinoma)

## 0. Making Pandas dataframe including only X, Y values needed by model 

In [1]:
import pandas as pd
import numpy as np
import h5py

### 0-1. Downloading and exploring 2 tsv files from Xenabrowser

In [2]:
df1 = pd.read_csv('TCGA-STAD.htseq_fpkm-uq_transposed.tsv', sep='\t')
df2 = pd.read_csv('TCGA-STAD.GDC_phenotype.tsv', sep='\t')

In [3]:
# 407명 기증자의 모든 유전자 위치에서의 mRNA 발현량  
df1

,Ensembl_ID,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,ENSG00000198242.12,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
0,TCGA-D7-5577-01A,0.000000,0.000000,17.440424,0.0,17.929620,18.847523,0.000000,10.845012,23.646398,...,0.0,11.483109,0.000000,13.398034,0.000000,19.959486,10.999693,0.00000,14.477365,0.000000
1,TCGA-D7-6818-01A,0.000000,7.328476,16.894902,0.0,16.047477,18.393746,0.000000,12.540158,22.206059,...,0.0,17.356251,0.000000,13.663386,0.000000,20.320092,13.067673,0.00000,14.652790,0.000000
2,TCGA-BR-7958-01A,0.000000,0.000000,16.724229,0.0,17.667541,18.118569,0.000000,10.439250,22.500445,...,0.0,9.883739,0.000000,13.037143,0.000000,19.314249,11.486559,0.00000,14.601858,0.000000
3,TCGA-D7-8572-01A,10.766176,8.242639,14.805945,0.0,17.222161,18.504515,10.098788,13.005000,22.130398,...,0.0,14.306127,9.840922,13.452516,11.824639,18.728202,12.738185,11.34276,13.842675,10.666970
4,TCGA-VQ-A91Z-01A,0.000000,9.877467,15.407496,0.0,15.795740,19.136198,0.000000,10.870857,22.254370,...,0.0,11.701971,0.000000,12.784554,11.375386,18.884585,14.448511,0.00000,12.932882,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,TCGA-CD-A489-01A,12.323254,0.000000,16.091254,0.0,16.427109,17.771472,0.000000,11.965715,22.719261,...,0.0,7.018427,0.000000,14.010101,12.475229,17.414922,12.388978,0.00000,13.694892,9.248802
403,TCGA-BR-6566-01A,0.000000,0.000000,16.719802,0.0,16.451107,18.733779,0.000000,10.719868,23.569497,...,0.0,14.880863,0.000000,13.370376,0.000000,19.882176,9.014762,0.00000,14.182239,0.000000
404,TCGA-VQ-A91S-01A,0.000000,6.564124,15.468126,0.0,16.259417,18.217679,0.000000,11.990096,21.839549,...,0.0,13.402292,0.000000,14.370714,10.136451,18.225644,10.857035,0.00000,12.693242,0.000000
405,TCGA-BR-8286-01A,10.967479,7.127704,16.180321,0.0,16.736216,17.407005,10.300027,12.500597,22.774849,...,0.0,11.728282,0.000000,14.823774,11.289263,19.140173,11.524837,0.00000,13.963588,11.230664


In [4]:
# 544명 기증자의 모든 GDC_phenotype 수치
df2

,submitter_id.samples,additional_pharmaceutical_therapy,additional_radiation_therapy,additional_surgery_locoregional_procedure,additional_surgery_metastatic_procedure,age_at_initial_pathologic_diagnosis,anatomic_neoplasm_subdivision_other,antireflux_treatment,barretts_esophagus,batch_number,...,intermediate_dimension.samples,is_ffpe.samples,longest_dimension.samples,oct_embedded.samples,preservation_method.samples,sample_type.samples,sample_type_id.samples,shortest_dimension.samples,state.samples,tissue_type.samples
0,TCGA-CG-5720-01A,NaN,NaN,NaN,NaN,71.0,NaN,NaN,No,95.85.0,...,0.6,False,0.9,NaN,NaN,Primary Tumor,1,0.6,released,Not Reported
1,TCGA-CG-5720-11A,NaN,NaN,NaN,NaN,71.0,NaN,NaN,No,95.85.0,...,0.8,False,1.6,NaN,NaN,Solid Tissue Normal,11,0.4,released,Not Reported
2,TCGA-VQ-A8PX-01A,NaN,NaN,NaN,NaN,51.0,Gastric stump,NO,No,427.44.0,...,NaN,False,NaN,False,NaN,Primary Tumor,1,NaN,released,Not Reported
3,TCGA-BR-A4CR-01A,NaN,NaN,NaN,NaN,70.0,NaN,NaN,NaN,257.69.0,...,NaN,False,NaN,True,NaN,Primary Tumor,1,NaN,released,Not Reported
4,TCGA-MX-A663-01A,NO,NO,NaN,NaN,66.0,Remnant gastrectomy after Billroth,NO,No,321.64.0,...,NaN,False,NaN,True,NaN,Primary Tumor,1,NaN,released,Not Reported
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,TCGA-HU-8243-11A,YES,NaN,NaN,NO,66.0,NaN,NaN,No,220.78.0,...,0.4,False,0.6,NaN,NaN,Solid Tissue Normal,11,0.3,released,Not Reported
540,TCGA-BR-8380-01A,NO,NO,NO,NaN,55.0,NaN,NaN,NaN,220.78.0,...,1.7,False,3.0,NaN,NaN,Primary Tumor,1,0.5,released,Not Reported
541,TCGA-BR-4183-01A,NaN,NaN,NaN,NaN,55.0,NaN,NO,NaN,48.87.0,...,1.4,False,3.0,NaN,NaN,Primary Tumor,1,0.5,released,Not Reported
542,TCGA-BR-4183-11A,NaN,NaN,NaN,NaN,55.0,NaN,NO,NaN,48.87.0,...,1.9,False,2.0,NaN,NaN,Solid Tissue Normal,11,0.4,released,Not Reported


### 0-2. Integrating two files into one using ID as key

In [5]:
# df1 과 df2 데이터의 기증자 수를 맞추려는 노력
df_x = df1.sort_values('Ensembl_ID')
df_x

,Ensembl_ID,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,ENSG00000198242.12,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
354,TCGA-3M-AB46-01A,9.172708,7.329795,17.563194,0.0,16.071659,17.679128,15.409254,13.847298,23.023774,...,0.0,13.609492,0.000000,17.223578,9.908674,18.538086,11.521835,0.0,13.611363,0.0
93,TCGA-3M-AB47-01A,0.000000,7.131909,15.942753,0.0,16.576146,18.984338,0.000000,11.992052,22.195397,...,0.0,16.931121,0.000000,15.015629,11.708390,18.432997,11.943986,0.0,13.446515,0.0
393,TCGA-B7-5818-01A,0.000000,0.000000,16.864073,0.0,17.136882,18.764149,0.000000,8.910704,23.446728,...,0.0,10.022942,0.000000,13.339954,0.000000,19.479232,12.583815,0.0,14.511396,0.0
270,TCGA-B7-A5TI-01A,8.878001,0.000000,16.116272,0.0,17.070254,18.343037,0.000000,9.386831,22.680389,...,0.0,13.837762,7.954758,14.021490,9.613739,18.773215,10.111896,0.0,13.480218,0.0
152,TCGA-B7-A5TJ-01A,0.000000,0.000000,15.842664,0.0,16.094085,18.994900,0.000000,12.888319,22.623489,...,0.0,15.372651,0.000000,14.087535,11.678386,18.732526,11.762020,0.0,13.115669,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,TCGA-VQ-AA6G-01A,0.000000,5.902859,16.295681,0.0,16.489625,18.041383,10.061425,12.960340,21.515821,...,0.0,12.648472,0.000000,12.415254,0.000000,18.653051,12.993535,0.0,12.379185,0.0
276,TCGA-VQ-AA6J-01A,8.274892,6.437561,15.846170,0.0,16.477069,18.576673,0.000000,13.072919,22.461277,...,0.0,12.254013,0.000000,14.911510,10.593089,18.660539,12.851858,0.0,15.844048,0.0
334,TCGA-VQ-AA6K-01A,8.824773,5.994980,16.614839,0.0,16.735731,19.042212,0.000000,11.447325,21.602157,...,0.0,11.534263,7.901634,14.900993,9.560465,19.659519,13.027048,0.0,13.707876,0.0
103,TCGA-ZA-A8F6-01A,8.974378,6.142665,16.158532,0.0,16.715901,18.215005,0.000000,11.724533,21.865307,...,0.0,16.064140,0.000000,15.016143,11.708904,18.120601,12.792334,0.0,13.561190,0.0


In [6]:
# phenotype 데이터 중 악성 종양 등급 데이터만 필요하여 골라 냄
df_y = df2[['submitter_id.samples','neoplasm_histologic_grade']].sort_values('submitter_id.samples')
df_y

,submitter_id.samples,neoplasm_histologic_grade
444,TCGA-3M-AB46-01A,G2
343,TCGA-3M-AB47-01A,G3
242,TCGA-B7-5816-01A,G3
38,TCGA-B7-5818-01A,G3
104,TCGA-B7-A5TI-01A,G3
...,...,...
141,TCGA-VQ-AA6I-01A,G3
519,TCGA-VQ-AA6J-01A,G3
479,TCGA-VQ-AA6K-01A,G3
308,TCGA-ZA-A8F6-01A,G2


In [7]:
# 기증자ID가 일치하는 행만 가지고 데이터프레임을 합침
df_INNER_JOIN = pd.merge(df_y, df_x, left_on='submitter_id.samples', right_on='Ensembl_ID', how='inner')
df_INNER_JOIN

,submitter_id.samples,neoplasm_histologic_grade,Ensembl_ID,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
0,TCGA-3M-AB46-01A,G2,TCGA-3M-AB46-01A,9.172708,7.329795,17.563194,0.0,16.071659,17.679128,15.409254,...,0.0,13.609492,0.000000,17.223578,9.908674,18.538086,11.521835,0.0,13.611363,0.0
1,TCGA-3M-AB47-01A,G3,TCGA-3M-AB47-01A,0.000000,7.131909,15.942753,0.0,16.576146,18.984338,0.000000,...,0.0,16.931121,0.000000,15.015629,11.708390,18.432997,11.943986,0.0,13.446515,0.0
2,TCGA-B7-5818-01A,G3,TCGA-B7-5818-01A,0.000000,0.000000,16.864073,0.0,17.136882,18.764149,0.000000,...,0.0,10.022942,0.000000,13.339954,0.000000,19.479232,12.583815,0.0,14.511396,0.0
3,TCGA-B7-A5TI-01A,G3,TCGA-B7-A5TI-01A,8.878001,0.000000,16.116272,0.0,17.070254,18.343037,0.000000,...,0.0,13.837762,7.954758,14.021490,9.613739,18.773215,10.111896,0.0,13.480218,0.0
4,TCGA-B7-A5TJ-01A,G1,TCGA-B7-A5TJ-01A,0.000000,0.000000,15.842664,0.0,16.094085,18.994900,0.000000,...,0.0,15.372651,0.000000,14.087535,11.678386,18.732526,11.762020,0.0,13.115669,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,TCGA-VQ-AA6G-01A,G2,TCGA-VQ-AA6G-01A,0.000000,5.902859,16.295681,0.0,16.489625,18.041383,10.061425,...,0.0,12.648472,0.000000,12.415254,0.000000,18.653051,12.993535,0.0,12.379185,0.0
403,TCGA-VQ-AA6J-01A,G3,TCGA-VQ-AA6J-01A,8.274892,6.437561,15.846170,0.0,16.477069,18.576673,0.000000,...,0.0,12.254013,0.000000,14.911510,10.593089,18.660539,12.851858,0.0,15.844048,0.0
404,TCGA-VQ-AA6K-01A,G3,TCGA-VQ-AA6K-01A,8.824773,5.994980,16.614839,0.0,16.735731,19.042212,0.000000,...,0.0,11.534263,7.901634,14.900993,9.560465,19.659519,13.027048,0.0,13.707876,0.0
405,TCGA-ZA-A8F6-01A,G2,TCGA-ZA-A8F6-01A,8.974378,6.142665,16.158532,0.0,16.715901,18.215005,0.000000,...,0.0,16.064140,0.000000,15.016143,11.708904,18.120601,12.792334,0.0,13.561190,0.0


In [8]:
# Ensembl_ID 컬럼은 중복되므로 떨구어 줌
df_xy_full = df_INNER_JOIN.drop('Ensembl_ID', axis=1)
df_xy_full

,submitter_id.samples,neoplasm_histologic_grade,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
0,TCGA-3M-AB46-01A,G2,9.172708,7.329795,17.563194,0.0,16.071659,17.679128,15.409254,13.847298,...,0.0,13.609492,0.000000,17.223578,9.908674,18.538086,11.521835,0.0,13.611363,0.0
1,TCGA-3M-AB47-01A,G3,0.000000,7.131909,15.942753,0.0,16.576146,18.984338,0.000000,11.992052,...,0.0,16.931121,0.000000,15.015629,11.708390,18.432997,11.943986,0.0,13.446515,0.0
2,TCGA-B7-5818-01A,G3,0.000000,0.000000,16.864073,0.0,17.136882,18.764149,0.000000,8.910704,...,0.0,10.022942,0.000000,13.339954,0.000000,19.479232,12.583815,0.0,14.511396,0.0
3,TCGA-B7-A5TI-01A,G3,8.878001,0.000000,16.116272,0.0,17.070254,18.343037,0.000000,9.386831,...,0.0,13.837762,7.954758,14.021490,9.613739,18.773215,10.111896,0.0,13.480218,0.0
4,TCGA-B7-A5TJ-01A,G1,0.000000,0.000000,15.842664,0.0,16.094085,18.994900,0.000000,12.888319,...,0.0,15.372651,0.000000,14.087535,11.678386,18.732526,11.762020,0.0,13.115669,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,TCGA-VQ-AA6G-01A,G2,0.000000,5.902859,16.295681,0.0,16.489625,18.041383,10.061425,12.960340,...,0.0,12.648472,0.000000,12.415254,0.000000,18.653051,12.993535,0.0,12.379185,0.0
403,TCGA-VQ-AA6J-01A,G3,8.274892,6.437561,15.846170,0.0,16.477069,18.576673,0.000000,13.072919,...,0.0,12.254013,0.000000,14.911510,10.593089,18.660539,12.851858,0.0,15.844048,0.0
404,TCGA-VQ-AA6K-01A,G3,8.824773,5.994980,16.614839,0.0,16.735731,19.042212,0.000000,11.447325,...,0.0,11.534263,7.901634,14.900993,9.560465,19.659519,13.027048,0.0,13.707876,0.0
405,TCGA-ZA-A8F6-01A,G2,8.974378,6.142665,16.158532,0.0,16.715901,18.215005,0.000000,11.724533,...,0.0,16.064140,0.000000,15.016143,11.708904,18.120601,12.792334,0.0,13.561190,0.0


### 0-3. Extracting Gene ID list related to STAD

In [12]:
# 위암과 관련 있는 유전자ID 리스트 (GASC 1202개에 PHGDH(ENSG00000092621)와 단백질 상호작용하는 유전자 369 를 더하고 ENSG ID가 없는 17개를 뺀 총 1554개)
df = pd.read_excel('GASCplusPHGDH_1554.xlsx')
df

,Gene Symbol,Ensembl ID
0,A2M-AS1,ENSG00000245105
1,AAR2,ENSG00000131043
2,ABCB1,ENSG00000085563
3,ABCC1,ENSG00000103222
4,ABCC2,ENSG00000023839
...,...,...
1549,ZNF280C,ENSG00000056277
1550,ZNF598,ENSG00000167962
1551,ZNF609,ENSG00000180357
1552,ZNF703,ENSG00000183779


### 0-4. Making a dataframe integrating 0-2 and 0-3

In [13]:
df['Ensembl ID'].count()

1554

In [14]:
df_xy_small = df_xy_full[['submitter_id.samples', 'neoplasm_histologic_grade']]
df_xy_small

,submitter_id.samples,neoplasm_histologic_grade
0,TCGA-3M-AB46-01A,G2
1,TCGA-3M-AB47-01A,G3
2,TCGA-B7-5818-01A,G3
3,TCGA-B7-A5TI-01A,G3
4,TCGA-B7-A5TJ-01A,G1
...,...,...
402,TCGA-VQ-AA6G-01A,G2
403,TCGA-VQ-AA6J-01A,G3
404,TCGA-VQ-AA6K-01A,G3
405,TCGA-ZA-A8F6-01A,G2


In [15]:
df['Ensembl ID'][41]

'ENSG00000211454'

In [16]:
df['Ensembl ID'].dropna().drop_duplicates()

0       ENSG00000245105
1       ENSG00000131043
2       ENSG00000085563
3       ENSG00000103222
4       ENSG00000023839
             ...       
1549    ENSG00000056277
1550    ENSG00000167962
1551    ENSG00000180357
1552    ENSG00000183779
1553    ENSG00000019995
Name: Ensembl ID, Length: 1554, dtype: object

In [ ]:
# 6만여개의 유전자리스트를 포함하는 전체 데이터프레임으로부터 위암과 관련있는 컬럼만을 뽑아 붙임 (53개는 그 원래 6만 개에 없어서 1554 - 53 = 1501 개만 남음)
for E_ID in df['Ensembl ID'].dropna().drop_duplicates():
    df_xy_small = pd.concat([df_xy_small, df_xy_full.filter(regex=E_ID)], axis=1)
df_xy_small

,submitter_id.samples,neoplasm_histologic_grade,ENSG00000245105.2,ENSG00000131043.10,ENSG00000085563.13,ENSG00000103222.17,ENSG00000023839.9,ENSG00000108846.14,ENSG00000091262.13,ENSG00000118777.9,...,ENSG00000177410.11,ENSG00000140836.13,ENSG00000253661.1,ENSG00000166432.13,ENSG00000163848.17,ENSG00000056277.14,ENSG00000167962.11,ENSG00000180357.8,ENSG00000183779.6,ENSG00000019995.6
0,TCGA-3M-AB46-01A,G2,12.631540,18.312495,14.318532,16.364047,11.037089,18.659929,15.430360,13.646748,...,21.045336,16.180944,11.412951,13.115407,17.260566,15.524566,17.837062,15.338686,19.374794,15.734897
1,TCGA-3M-AB47-01A,G3,12.959527,17.934268,16.160946,18.282712,11.768339,16.547711,15.367687,16.074496,...,19.789973,16.952844,12.891470,14.712015,16.866016,15.622206,18.062104,16.553608,21.262279,15.886567
2,TCGA-B7-5818-01A,G3,9.805638,18.645617,12.880046,18.169850,7.718481,19.585772,11.018773,10.981525,...,19.983948,16.317194,8.185707,10.143445,17.138078,15.401180,18.617195,17.257773,18.325260,15.495983
3,TCGA-B7-A5TI-01A,G3,13.179467,18.243948,15.407460,18.950430,15.200596,20.725672,16.640060,13.642704,...,19.624785,15.496281,9.534131,11.914558,16.778476,14.715512,18.427179,17.038676,18.098576,15.567411
4,TCGA-B7-A5TJ-01A,G1,13.172761,18.771969,14.176191,18.124846,14.429294,19.441774,15.288600,13.754549,...,20.980256,17.172219,9.862818,11.560136,17.101747,14.773824,19.187604,16.485147,21.817616,15.463538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,TCGA-VQ-AA6G-01A,G2,11.189674,17.880143,13.535621,17.766386,10.952426,19.858542,15.618313,13.916546,...,18.220067,16.016099,0.000000,11.106274,17.097017,14.255276,18.389971,17.379477,19.943897,15.953224
403,TCGA-VQ-AA6J-01A,G3,13.843641,18.303575,14.388388,18.411861,15.848974,19.732003,15.591814,16.345803,...,19.652691,16.994635,8.609251,15.553067,17.214653,15.752475,18.069899,16.968723,20.527869,15.465501
404,TCGA-VQ-AA6K-01A,G3,13.195385,18.808490,15.105125,18.414490,17.505974,19.725012,15.973455,14.169108,...,19.427959,16.877440,10.742717,14.342780,17.279563,14.780608,18.611692,17.048796,20.313476,15.819704
405,TCGA-ZA-A8F6-01A,G2,14.989051,18.202771,18.111348,18.294249,13.809347,18.927904,15.845785,16.732377,...,19.494675,17.223242,11.699583,14.811041,16.962942,15.529613,17.571429,17.048029,20.722353,15.953487


In [19]:
df_xy_small.to_pickle('xy_STAD-related-only3.pkl')
df_xy_small.to_csv('xy_STAD-related-only3.csv')

### 0-4-1. Double checking whether PHDGH & SFRP1,2,4,5 are inclued in newly made dataframe

In [20]:
df_xy_small.columns

Index(['submitter_id.samples', 'neoplasm_histologic_grade',
       'ENSG00000245105.2', 'ENSG00000131043.10', 'ENSG00000085563.13',
       'ENSG00000103222.17', 'ENSG00000023839.9', 'ENSG00000108846.14',
       'ENSG00000091262.13', 'ENSG00000118777.9',
       ...
       'ENSG00000177410.11', 'ENSG00000140836.13', 'ENSG00000253661.1',
       'ENSG00000166432.13', 'ENSG00000163848.17', 'ENSG00000056277.14',
       'ENSG00000167962.11', 'ENSG00000180357.8', 'ENSG00000183779.6',
       'ENSG00000019995.6'],
      dtype='object', length=1503)

In [ ]:
# 이번에 새로 만든 1501 개에 PHGDH 가 포함되어 있는 것 확인 
for ID in df_xy_small.columns:
    if 'ENSG00000092621' in ID:
        print("PHGDH included :", ID)

PHGDH included : ENSG00000092621.10


In [ ]:
# 이번에 새로 만든 1501 개에 SFRP1 이 포함되어 있는 것 확인 
for ID in df_xy_small.columns:
    if 'ENSG00000104332' in ID:
        print("SFRP1 included :", ID)

SFRP1 included : ENSG00000104332.10


In [ ]:
# 이번에 새로 만든 1501 개에 SFRP2 이 포함되어 있는 것 확인 
for ID in df_xy_small.columns:
    if 'ENSG00000145423' in ID:
        print("SFRP2 included :", ID)

SFRP2 included : ENSG00000145423.4


In [ ]:
# 이번에 새로 만든 1501 개에 SFRP4 이 포함되어 있는 것 확인 
for ID in df_xy_small.columns:
    if 'ENSG00000106483' in ID:
        print("SFRP4 included :", ID)

SFRP4 included : ENSG00000106483.10


In [ ]:
# 이번에 새로 만든 1501 개에 SFRP5 이 포함되어 있는 것 확인 
for ID in df_xy_small.columns:
    if 'ENSG00000120057' in ID:
        print("SFRP5 included :", ID)

SFRP5 included : ENSG00000120057.4


## 1. Making the final numpy dataset of XY values for inserting into model

In [2]:
import pandas as pd
import numpy as np
import h5py

### 1-1. Trim & Sort

In [34]:
df = pd.read_pickle('xy_STAD-related-only3.pkl')
df

,submitter_id.samples,neoplasm_histologic_grade,ENSG00000245105.2,ENSG00000131043.10,ENSG00000085563.13,ENSG00000103222.17,ENSG00000023839.9,ENSG00000108846.14,ENSG00000091262.13,ENSG00000118777.9,...,ENSG00000177410.11,ENSG00000140836.13,ENSG00000253661.1,ENSG00000166432.13,ENSG00000163848.17,ENSG00000056277.14,ENSG00000167962.11,ENSG00000180357.8,ENSG00000183779.6,ENSG00000019995.6
0,TCGA-3M-AB46-01A,G2,12.631540,18.312495,14.318532,16.364047,11.037089,18.659929,15.430360,13.646748,...,21.045336,16.180944,11.412951,13.115407,17.260566,15.524566,17.837062,15.338686,19.374794,15.734897
1,TCGA-3M-AB47-01A,G3,12.959527,17.934268,16.160946,18.282712,11.768339,16.547711,15.367687,16.074496,...,19.789973,16.952844,12.891470,14.712015,16.866016,15.622206,18.062104,16.553608,21.262279,15.886567
2,TCGA-B7-5818-01A,G3,9.805638,18.645617,12.880046,18.169850,7.718481,19.585772,11.018773,10.981525,...,19.983948,16.317194,8.185707,10.143445,17.138078,15.401180,18.617195,17.257773,18.325260,15.495983
3,TCGA-B7-A5TI-01A,G3,13.179467,18.243948,15.407460,18.950430,15.200596,20.725672,16.640060,13.642704,...,19.624785,15.496281,9.534131,11.914558,16.778476,14.715512,18.427179,17.038676,18.098576,15.567411
4,TCGA-B7-A5TJ-01A,G1,13.172761,18.771969,14.176191,18.124846,14.429294,19.441774,15.288600,13.754549,...,20.980256,17.172219,9.862818,11.560136,17.101747,14.773824,19.187604,16.485147,21.817616,15.463538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,TCGA-VQ-AA6G-01A,G2,11.189674,17.880143,13.535621,17.766386,10.952426,19.858542,15.618313,13.916546,...,18.220067,16.016099,0.000000,11.106274,17.097017,14.255276,18.389971,17.379477,19.943897,15.953224
403,TCGA-VQ-AA6J-01A,G3,13.843641,18.303575,14.388388,18.411861,15.848974,19.732003,15.591814,16.345803,...,19.652691,16.994635,8.609251,15.553067,17.214653,15.752475,18.069899,16.968723,20.527869,15.465501
404,TCGA-VQ-AA6K-01A,G3,13.195385,18.808490,15.105125,18.414490,17.505974,19.725012,15.973455,14.169108,...,19.427959,16.877440,10.742717,14.342780,17.279563,14.780608,18.611692,17.048796,20.313476,15.819704
405,TCGA-ZA-A8F6-01A,G2,14.989051,18.202771,18.111348,18.294249,13.809347,18.927904,15.845785,16.732377,...,19.494675,17.223242,11.699583,14.811041,16.962942,15.529613,17.571429,17.048029,20.722353,15.953487


In [35]:
df.columns[2:]

Index(['ENSG00000245105.2', 'ENSG00000131043.10', 'ENSG00000085563.13',
       'ENSG00000103222.17', 'ENSG00000023839.9', 'ENSG00000108846.14',
       'ENSG00000091262.13', 'ENSG00000118777.9', 'ENSG00000175164.12',
       'ENSG00000117054.12',
       ...
       'ENSG00000177410.11', 'ENSG00000140836.13', 'ENSG00000253661.1',
       'ENSG00000166432.13', 'ENSG00000163848.17', 'ENSG00000056277.14',
       'ENSG00000167962.11', 'ENSG00000180357.8', 'ENSG00000183779.6',
       'ENSG00000019995.6'],
      dtype='object', length=1501)

In [37]:
# 유전자아이디에서 점이하를 버린 리스트를 만들고
gene_ids_trimmed = [gene_id[:15] for gene_id in df.columns[2:]]
gene_ids_trimmed[:5]

['ENSG00000245105',
 'ENSG00000131043',
 'ENSG00000085563',
 'ENSG00000103222',
 'ENSG00000023839']

In [38]:
# 그 리스트를 가지고 데이터프레임을 다시 만듦
for original, trimmed in zip(df.columns[2:], gene_ids_trimmed):
    df.rename(columns={original:trimmed}, inplace=True)
df

,submitter_id.samples,neoplasm_histologic_grade,ENSG00000245105,ENSG00000131043,ENSG00000085563,ENSG00000103222,ENSG00000023839,ENSG00000108846,ENSG00000091262,ENSG00000118777,...,ENSG00000177410,ENSG00000140836,ENSG00000253661,ENSG00000166432,ENSG00000163848,ENSG00000056277,ENSG00000167962,ENSG00000180357,ENSG00000183779,ENSG00000019995
0,TCGA-3M-AB46-01A,G2,12.631540,18.312495,14.318532,16.364047,11.037089,18.659929,15.430360,13.646748,...,21.045336,16.180944,11.412951,13.115407,17.260566,15.524566,17.837062,15.338686,19.374794,15.734897
1,TCGA-3M-AB47-01A,G3,12.959527,17.934268,16.160946,18.282712,11.768339,16.547711,15.367687,16.074496,...,19.789973,16.952844,12.891470,14.712015,16.866016,15.622206,18.062104,16.553608,21.262279,15.886567
2,TCGA-B7-5818-01A,G3,9.805638,18.645617,12.880046,18.169850,7.718481,19.585772,11.018773,10.981525,...,19.983948,16.317194,8.185707,10.143445,17.138078,15.401180,18.617195,17.257773,18.325260,15.495983
3,TCGA-B7-A5TI-01A,G3,13.179467,18.243948,15.407460,18.950430,15.200596,20.725672,16.640060,13.642704,...,19.624785,15.496281,9.534131,11.914558,16.778476,14.715512,18.427179,17.038676,18.098576,15.567411
4,TCGA-B7-A5TJ-01A,G1,13.172761,18.771969,14.176191,18.124846,14.429294,19.441774,15.288600,13.754549,...,20.980256,17.172219,9.862818,11.560136,17.101747,14.773824,19.187604,16.485147,21.817616,15.463538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,TCGA-VQ-AA6G-01A,G2,11.189674,17.880143,13.535621,17.766386,10.952426,19.858542,15.618313,13.916546,...,18.220067,16.016099,0.000000,11.106274,17.097017,14.255276,18.389971,17.379477,19.943897,15.953224
403,TCGA-VQ-AA6J-01A,G3,13.843641,18.303575,14.388388,18.411861,15.848974,19.732003,15.591814,16.345803,...,19.652691,16.994635,8.609251,15.553067,17.214653,15.752475,18.069899,16.968723,20.527869,15.465501
404,TCGA-VQ-AA6K-01A,G3,13.195385,18.808490,15.105125,18.414490,17.505974,19.725012,15.973455,14.169108,...,19.427959,16.877440,10.742717,14.342780,17.279563,14.780608,18.611692,17.048796,20.313476,15.819704
405,TCGA-ZA-A8F6-01A,G2,14.989051,18.202771,18.111348,18.294249,13.809347,18.927904,15.845785,16.732377,...,19.494675,17.223242,11.699583,14.811041,16.962942,15.529613,17.571429,17.048029,20.722353,15.953487


In [39]:
# 컬럼 이름 오름차순으로 데이터프레임 정렬(나중에 새로운 데이터로 추론시에 유전자아이디가 섞여 있으면 안 되기 때문)
dfs = df.sort_index(axis=1)
dfs

,ENSG00000001036,ENSG00000001626,ENSG00000002330,ENSG00000003400,ENSG00000003402,ENSG00000005007,ENSG00000005156,ENSG00000005175,ENSG00000005194,ENSG00000005844,...,ENSG00000278910,ENSG00000280924,ENSG00000280997,ENSG00000281183,ENSG00000281189,ENSG00000281202,ENSG00000281406,ENSG00000281450,neoplasm_histologic_grade,submitter_id.samples
0,20.220979,14.665292,18.727956,18.054834,16.883271,17.875547,16.233214,17.925067,17.105839,14.004341,...,9.415178,13.167983,0.000000,13.485292,13.578557,0.000000,15.438352,8.090678,G2,TCGA-3M-AB46-01A
1,18.500387,18.794606,18.090443,17.441156,17.084666,18.313232,15.072061,16.966393,18.010680,16.427446,...,10.799621,11.281136,0.000000,14.944210,11.546818,0.000000,12.354517,8.889208,G3,TCGA-3M-AB47-01A
2,19.761280,19.947795,19.709574,16.382809,17.009260,18.794353,16.590639,16.450929,18.271574,17.263513,...,11.088938,14.599826,8.546172,12.620040,13.064786,0.000000,15.455768,9.762250,G3,TCGA-B7-5818-01A
3,19.181518,10.613326,18.942790,16.730176,17.504471,18.283692,15.722849,16.700315,18.246164,15.791234,...,10.703615,13.796031,7.579382,13.917905,11.974194,0.000000,16.328196,0.000000,G3,TCGA-B7-A5TI-01A
4,19.586652,17.520498,18.527097,16.876615,16.508963,18.872212,15.133784,17.002714,18.221248,14.314973,...,10.185077,12.604413,10.960521,14.445830,12.838448,9.294717,16.457270,0.000000,G1,TCGA-B7-A5TJ-01A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,18.996392,16.725651,17.717748,19.014720,17.760508,19.068443,16.721096,16.808091,18.048495,14.208590,...,8.973679,13.890051,7.433210,14.811144,12.551461,0.000000,15.849008,9.230485,G2,TCGA-VQ-AA6G-01A
403,18.794649,17.282670,17.603730,17.640446,17.107674,18.107977,16.429840,16.958818,18.187556,17.889670,...,10.099362,14.217296,8.969995,14.709362,13.369268,0.000000,15.227504,9.187872,G3,TCGA-VQ-AA6J-01A
404,19.295330,18.157491,17.706194,17.364764,17.483705,18.411735,16.221507,16.245028,18.307993,16.396496,...,11.649856,11.868503,10.104749,15.562842,12.255207,0.000000,17.192497,8.740135,G3,TCGA-VQ-AA6K-01A
405,18.647983,17.995544,18.393344,17.702167,17.343445,18.255709,15.532520,16.528308,17.310203,16.494008,...,10.215577,11.918873,0.000000,15.733577,12.591478,0.000000,14.480349,0.000000,G2,TCGA-ZA-A8F6-01A


In [40]:
# 정답값이 되는 악성 종양 등급 별로 행 정렬(훈련,검증데이터로 나눌 때 필요)
dfs2 = dfs.sort_values(by='neoplasm_histologic_grade')
dfs2

,ENSG00000001036,ENSG00000001626,ENSG00000002330,ENSG00000003400,ENSG00000003402,ENSG00000005007,ENSG00000005156,ENSG00000005175,ENSG00000005194,ENSG00000005844,...,ENSG00000278910,ENSG00000280924,ENSG00000280997,ENSG00000281183,ENSG00000281189,ENSG00000281202,ENSG00000281406,ENSG00000281450,neoplasm_histologic_grade,submitter_id.samples
318,20.219629,16.825263,19.121479,17.287974,16.630257,17.568893,15.572426,16.993762,18.063461,15.244971,...,11.962227,13.237309,8.099240,15.439994,12.557625,0.000000,14.470708,0.000000,G1,TCGA-KB-A93H-01A
317,18.819095,12.144605,17.963353,16.261201,18.064107,18.698039,15.809532,16.613230,17.294052,15.892170,...,0.000000,11.723755,10.902820,15.902137,11.159664,0.000000,13.233482,8.801656,G1,TCGA-KB-A93G-01A
132,19.807727,17.048517,19.515146,17.236227,16.513151,18.753454,16.319948,16.776418,18.327796,15.494267,...,0.000000,13.746728,8.880671,13.426548,14.626484,0.000000,16.505431,0.000000,G1,TCGA-CD-5800-01A
179,18.941061,15.740072,19.171449,16.252021,17.308548,18.667955,15.786658,16.111774,18.430531,16.296577,...,0.000000,13.519727,0.000000,15.516455,12.852409,7.879561,16.408713,8.441310,G1,TCGA-CG-5719-01A
258,19.919778,18.664863,18.456981,17.296716,16.551752,18.715583,16.050815,17.181970,17.770386,13.915716,...,0.000000,12.874540,9.264021,15.442020,13.462183,0.000000,14.359035,0.000000,G1,TCGA-HU-8244-01A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,18.749838,14.442284,18.851856,16.423086,17.029071,18.415530,16.069512,17.105799,17.426681,15.877429,...,11.912352,11.394259,7.374729,15.455789,12.198967,7.031628,13.265510,10.392719,GX,TCGA-BR-8297-01A
378,20.160689,19.391947,17.841738,17.806221,17.092400,18.839794,16.161789,17.072886,17.125779,16.660077,...,11.641512,10.708630,11.487912,14.583587,13.082237,0.000000,15.675195,10.314506,GX,TCGA-VQ-A91S-01A
344,19.631296,11.517487,18.765269,17.731691,17.073518,18.193675,15.092689,16.983539,17.902584,15.178758,...,13.597188,12.146249,0.000000,15.329461,9.124790,0.000000,14.680185,10.784776,GX,TCGA-SW-A7EA-01A
50,19.331497,17.675182,18.370031,16.306453,17.176000,20.383370,15.860547,16.679588,18.546336,14.489921,...,10.363710,12.166602,0.000000,14.522243,13.625953,9.794749,14.882827,0.000000,GX,TCGA-BR-7197-01A


### 1-2. Extracting dataset X & Y

In [41]:
df_X = dfs2.drop(labels=['neoplasm_histologic_grade','submitter_id.samples'], axis=1)
df_X

,ENSG00000001036,ENSG00000001626,ENSG00000002330,ENSG00000003400,ENSG00000003402,ENSG00000005007,ENSG00000005156,ENSG00000005175,ENSG00000005194,ENSG00000005844,...,ENSG00000278637,ENSG00000278705,ENSG00000278910,ENSG00000280924,ENSG00000280997,ENSG00000281183,ENSG00000281189,ENSG00000281202,ENSG00000281406,ENSG00000281450
318,20.219629,16.825263,19.121479,17.287974,16.630257,17.568893,15.572426,16.993762,18.063461,15.244971,...,14.168395,14.486349,11.962227,13.237309,8.099240,15.439994,12.557625,0.000000,14.470708,0.000000
317,18.819095,12.144605,17.963353,16.261201,18.064107,18.698039,15.809532,16.613230,17.294052,15.892170,...,0.000000,11.902443,0.000000,11.723755,10.902820,15.902137,11.159664,0.000000,13.233482,8.801656
132,19.807727,17.048517,19.515146,17.236227,16.513151,18.753454,16.319948,16.776418,18.327796,15.494267,...,13.952046,13.877703,0.000000,13.746728,8.880671,13.426548,14.626484,0.000000,16.505431,0.000000
179,18.941061,15.740072,19.171449,16.252021,17.308548,18.667955,15.786658,16.111774,18.430531,16.296577,...,13.293363,11.804238,0.000000,13.519727,0.000000,15.516455,12.852409,7.879561,16.408713,8.441310
258,19.919778,18.664863,18.456981,17.296716,16.551752,18.715583,16.050815,17.181970,17.770386,13.915716,...,13.073153,14.168642,0.000000,12.874540,9.264021,15.442020,13.462183,0.000000,14.359035,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,18.749838,14.442284,18.851856,16.423086,17.029071,18.415530,16.069512,17.105799,17.426681,15.877429,...,10.856173,11.951336,11.912352,11.394259,7.374729,15.455789,12.198967,7.031628,13.265510,10.392719
378,20.160689,19.391947,17.841738,17.806221,17.092400,18.839794,16.161789,17.072886,17.125779,16.660077,...,10.585416,10.680933,11.641512,10.708630,11.487912,14.583587,13.082237,0.000000,15.675195,10.314506
344,19.631296,11.517487,18.765269,17.731691,17.073518,18.193675,15.092689,16.983539,17.902584,15.178758,...,14.192642,15.858483,13.597188,12.146249,0.000000,15.329461,9.124790,0.000000,14.680185,10.784776
50,19.331497,17.675182,18.370031,16.306453,17.176000,20.383370,15.860547,16.679588,18.546336,14.489921,...,14.554019,14.571593,10.363710,12.166602,0.000000,14.522243,13.625953,9.794749,14.882827,0.000000


In [42]:
df_X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 407 entries, 318 to 66
Columns: 1501 entries, ENSG00000001036 to ENSG00000281450
dtypes: float64(1501)
memory usage: 4.7 MB


In [43]:
df_Y = dfs2[['neoplasm_histologic_grade']]
df_Y

,neoplasm_histologic_grade
318,G1
317,G1
132,G1
179,G1
258,G1
...,...
82,GX
378,GX
344,GX
50,GX


In [44]:
df_Y.info()

<class 'pandas.core.frame.DataFrame'>
Index: 407 entries, 318 to 66
Data columns (total 1 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   neoplasm_histologic_grade  407 non-null    object
dtypes: object(1)
memory usage: 6.4+ KB


In [45]:
df_Y.shape

(407, 1)

In [46]:
print('G1:  ', len(df_Y[df_Y['neoplasm_histologic_grade']=='G1']))
print('G2: ', len(df_Y[df_Y['neoplasm_histologic_grade']=='G2']))
print('G3: ', len(df_Y[df_Y['neoplasm_histologic_grade']=='G3']))
print('GX:   ', len(df_Y[df_Y['neoplasm_histologic_grade']=='GX']))

G1:   10
G2:  150
G3:  238
GX:    9


In [47]:
# NLL Loss 계산 시 문제가 생겨 label 차원은 2차원 행렬이 아닌 평범한 1차원 벡터가 되게 바꿈 
df_Y = dfs2['neoplasm_histologic_grade']
df_Y

318    G1
317    G1
132    G1
179    G1
258    G1
       ..
82     GX
378    GX
344    GX
50     GX
66     GX
Name: neoplasm_histologic_grade, Length: 407, dtype: object

### 1-3. Splitting dataset into train, val, test 

In [48]:
# X, Y 데이터를 약 7 : 2 : 1 의 비율로 나눔
df_X_train = pd.concat([df_X[0: 7], df_X[ 10:115], df_X[160:327], df_X[398:404]])
df_X_val   = pd.concat([df_X[7: 9], df_X[115:145], df_X[327:375], df_X[404:406]])
df_X_test  = pd.concat([df_X[9:10], df_X[145:160], df_X[375:398], df_X[406:407]])
df_Y_train = pd.concat([df_Y[0: 7], df_Y[ 10:115], df_Y[160:327], df_Y[398:404]])
df_Y_val   = pd.concat([df_Y[7: 9], df_Y[115:145], df_Y[327:375], df_Y[404:406]])
df_Y_test  = pd.concat([df_Y[9:10], df_Y[145:160], df_Y[375:398], df_Y[406:407]])

In [49]:
len(df_Y_train)+len(df_Y_val)+len(df_Y_test)

407

In [50]:
# X 데이터 : pandas dataframe 형식에서 numpy array 형식으로 변환
X_train = df_X_train.to_numpy(dtype='float64')
X_val = df_X_val.to_numpy(dtype='float64')
X_test = df_X_test.to_numpy(dtype='float64')
X_train.shape, X_train[0].dtype

((285, 1501), dtype('float64'))

In [51]:
# Y 데이터 : 문자에서 정수로 변환 G1->1, G2->2, G3->3, GX->0
Y_train = df_Y_train.replace({'G1':1, 'G2':2, 'G3':3, 'GX':0}).to_numpy(dtype='int8')
Y_val = df_Y_val.replace({'G1':1, 'G2':2, 'G3':3, 'GX':0}).to_numpy(dtype='int8')
Y_test = df_Y_test.replace({'G1':1, 'G2':2, 'G3':3, 'GX':0}).to_numpy(dtype='int8')
Y_train.shape, Y_train[0].dtype

((285,), dtype('int8'))

### 1-4. Min-Max Normalization for richer transformer embedding

In [52]:
np.min(X_train), np.max(X_train)

(0.0, 31.18721908083413)

In [53]:
def MinMax(X):
    X_norm = (X-np.min(X)) / (np.max(X)-np.min(X))
    return X_norm

In [54]:
np.min(MinMax(X_train)), np.max(MinMax(X_train))

(0.0, 1.0)

In [55]:
X_train_norm = MinMax(X_train) * 319
X_train_norm

array([[206.81746899, 172.09803197, 195.58499015, ...,   0.        ,
        148.0143489 ,   0.        ],
       [192.49203866, 124.22168367, 183.73903337, ...,   0.        ,
        135.35931518,  90.02817139],
       [202.60430677, 174.38158934, 199.61163048, ...,   0.        ,
        168.82660949,   0.        ],
       ...,
       [188.07301827, 142.69752987, 188.12953123, ..., 150.55937897,
        125.4813735 ,  79.84250847],
       [191.78363413, 147.72360726, 192.82713338, ...,  71.92334842,
        135.68691846, 106.30243534],
       [206.21459739, 198.3514809 , 182.49509247, ...,   0.        ,
        160.33449735, 105.50243126]])

In [56]:
np.min(X_train_norm), np.max(X_train_norm)

(0.0, 319.0)

In [57]:
X_train_norm = MinMax(X_train) * 319
X_val_norm = MinMax(X_val) * 319
X_test_norm = MinMax(X_test) * 319

### 1-5. Save datasets in 2 different ways

In [ ]:
# # 1. train:val:test 가 7:2:1 로 나뉜 데이터를 전부 가지고 있는 파일 (추후에 사용 예정, 현재는 불필요)
# hf = h5py.File('STAD_Dataset_split_in_three_expanded.h5', 'w')
# hf.create_dataset('X_train', data=X_train_norm)
# hf.create_dataset('X_val', data=X_val_norm)
# hf.create_dataset('X_test', data=X_test_norm)
# hf.create_dataset('Y_train', data=Y_train)
# hf.create_dataset('Y_val', data=Y_val)
# hf.create_dataset('Y_test', data=Y_test)
# hf.close()

In [58]:
X_Train = np.concatenate([X_train_norm, X_test_norm])
X_Train.shape

(325, 1501)

In [59]:
Y_Train = np.concatenate([Y_train, Y_test])
Y_Train.shape

(325,)

In [60]:
# 2. 위 train+test 를 합친 전체의 80% 인 Train 데이터파일 
hf = h5py.File('STAD_Dataset3_Train.h5', 'w')
hf.create_dataset('X_Train', data=X_Train)
hf.create_dataset('Y_Train', data=Y_Train)
hf.close()

In [61]:
# 3. 나머지 20% 만 가진 Test 데이터파일 
hf = h5py.File('STAD_Dataset3_Test.h5', 'w')
hf.create_dataset('X_Test', data=X_val_norm)
hf.create_dataset('Y_Test', data=Y_val)
hf.close()

### 1-6. Dataset Validation Check

In [ ]:
# hf = h5py.File('STAD_Dataset_split_in_three.h5', 'r')
# # hf.keys()
# X_train = np.array(hf.get('X_train'))
# X_val = np.array(hf.get('X_val'))
# X_test = np.array(hf.get('X_test'))
# Y_train = np.array(hf.get('Y_train'))
# Y_val = np.array(hf.get('Y_val'))
# Y_test = np.array(hf.get('Y_test'))

# print('X_train.shape : ', X_train.shape)
# print('X_val.shape   : ', X_val.shape)
# print('X_test.shape  : ', X_test.shape)
# print('Y_train.shape : ', Y_train.shape)
# print('Y_val.shape   : ', Y_val.shape)
# print('Y_test.shape  : ', Y_test.shape)

# hf.close()

X_train.shape :  (285, 843)
X_val.shape   :  (82, 843)
X_test.shape  :  (40, 843)
Y_train.shape :  (285,)
Y_val.shape   :  (82,)
Y_test.shape  :  (40,)


In [62]:
hf = h5py.File('STAD_Dataset3_Train.h5', 'r')

X_Train = np.array(hf.get('X_Train'))
Y_Train = np.array(hf.get('Y_Train'))
print(hf.keys())
print('X_Train.shape : ', X_Train.shape)
print('Y_Train.shape : ', Y_Train.shape)
print('X_Train' in hf.keys())

hf.close()

<KeysViewHDF5 ['X_Train', 'Y_Train']>
X_Train.shape :  (325, 1501)
Y_Train.shape :  (325,)
True


In [63]:
hf = h5py.File('STAD_Dataset3_Test.h5', 'r')
# hf.keys()
X_Test = np.array(hf.get('X_Test'))
Y_Test = np.array(hf.get('Y_Test'))

print('X_Test.shape  : ', X_Test.shape)
print('Y_Test.shape  : ', Y_Test.shape)

hf.close()

X_Test.shape  :  (82, 1501)
Y_Test.shape  :  (82,)


In [64]:
X_Train[0].dtype, Y_Train[0].dtype

(dtype('<f8'), dtype('int8'))